In [1]:
import pickle
import networkx as nx
from course_version2 import *

G28: after frocing the student to the best section
g29: before forcing the student to the best section

In [2]:
with open("real_graph.pickle", "rb") as f:
    G28 = pickle.load(f)
with open("test_graph.pickle", "rb") as f:
    G29 = pickle.load(f)

In [3]:
# create a copy of the graphs with all nodes as strings instead of sections
G28 = nx.relabel_nodes(G28, {i: i.name + "-" + str(i.block) for i in G28.nodes() if isinstance(i, Section)})
G29 = nx.relabel_nodes(G29, {i: i.name + "-" + str(i.block) for i in G29.nodes() if isinstance(i, Section)})


In [4]:
print(G28)
print(G29)
print("G28 is isomorphic to G29:", nx.is_isomorphic(G28, G29))

# compare the max flow min cost of the two

flow_dict28 = nx.max_flow_min_cost(G28, "source", "sink")
flow_dict29 = nx.max_flow_min_cost(G29, "source", "sink")

g28_edges = G28.edges(data=True)

DiGraph with 27 nodes and 48 edges
DiGraph with 27 nodes and 61 edges
G28 is isomorphic to G29: False


In [5]:
print(flow_dict28 == flow_dict29)
# find the difference between the dictionaries
for key in flow_dict28.keys():
    if key not in flow_dict29.keys():
        print("Key", key, "is not in G29")
    else:
        for sub_key in flow_dict28[key].keys():
            if sub_key not in flow_dict29[key].keys():
                print("Key", key, "sub_key", sub_key, "is not in G29")
            else:
                if flow_dict28[key][sub_key] != flow_dict29[key][sub_key]:
                    print("Key", key, "sub_key", sub_key, "is different")
                    print("G28:", flow_dict28[key][sub_key])	
                    print("G29:", flow_dict29[key][sub_key])
                    print("*****")

False


In [6]:
def value_of_flow(flow_dict: dict, G: nx.DiGraph) -> int:
    weighted_flow = 0
    unweighted_flow = sum((flow_dict[u]["sink"] for u in G.predecessors("sink"))) - sum(flow_dict["sink"][v] for v in G.successors("sink"))

    for u, v in flow_dict.items():
        for w, flow in v.items():
            if flow > 0:
                if "weight" in G[u][w].keys():
                    weighted_flow += G[u][w]["weight"] * flow
            # print(w, flow)
            # print(G[u][w])
    
    return weighted_flow, unweighted_flow

print(value_of_flow(flow_dict28, G28))
print(nx.maximum_flow_value(G28, "source", "sink"))

(2300, 23)
23


In [7]:
section3_students29 = set()
section5_student29 = set()
# compare the flow values
for student, all_flow in flow_dict29.items():
    if student == "source" or student == "sink":
        continue
    for section, flow_value in all_flow.items():
        if section[:3] == "MAT":
            if flow_value > 0:
                if section[6] == "3":
                    section3_students29.add(student)
                    # print(student, section, flow_value)
                else:
                    section5_student29.add(student)
                # print(student, section, flow_value)

print(section3_students29)
print(section5_student29)

{'Student 32 Grade 9', 'Student 45 Grade 9', 'Student 22 Grade 9', 'Student 50 Grade 9', 'Student 27 Grade 9', 'Student 19 Grade 9', 'Student 23 Grade 9', 'Student 33 Grade 9', 'Student 38 Grade 9', 'Student 26 Grade 9', 'Student 36 Grade 9', 'Student 43 Grade 9'}
{'Student 18 Grade 9', 'Student 34 Grade 9', 'Student 1 Grade 9', 'Student 31 Grade 9', 'Student 14 Grade 9', 'Student 12 Grade 9', 'Student 41 Grade 9', 'Student 17 Grade 9', 'Student 49 Grade 9', 'Student 24 Grade 9', 'Student 10 Grade 9'}


In [8]:
section3_students28 = set()
section5_student28 = set()
# compare the flow values
for student, all_flow in flow_dict28.items():
    if student == "source" or student == "sink":
        continue
    for section, flow_value in all_flow.items():
        if section[:3] == "MAT":
            if flow_value > 0:
                if section[6] == "3":
                    section3_students28.add(student)
                    # print(student, section, flow_value)
                else:
                    section5_student28.add(student)
                # print(student, section, flow_value)

print(section3_students28)
print(section5_student28)

{'Student 32 Grade 9', 'Student 45 Grade 9', 'Student 22 Grade 9', 'Student 50 Grade 9', 'Student 27 Grade 9', 'Student 19 Grade 9', 'Student 23 Grade 9', 'Student 33 Grade 9', 'Student 38 Grade 9', 'Student 26 Grade 9', 'Student 36 Grade 9', 'Student 43 Grade 9'}
{'Student 18 Grade 9', 'Student 34 Grade 9', 'Student 1 Grade 9', 'Student 31 Grade 9', 'Student 14 Grade 9', 'Student 12 Grade 9', 'Student 41 Grade 9', 'Student 17 Grade 9', 'Student 49 Grade 9', 'Student 24 Grade 9', 'Student 10 Grade 9'}


Comparing the values and costs of flows between G28 and G29 with the same flowdict (flowdit28).

To fix:
Force the alogorithm to do the same thing as the previous best iteration when it placed the student in that section.

In [9]:
print("G28 cost and value:", value_of_flow(flow_dict28, G28))
print("G29 cost and value:", value_of_flow(flow_dict29, G29))
print("G29 cost and value using G28:", value_of_flow(flow_dict29, G28))

G28 cost and value: (2300, 23)
G29 cost and value: (1090, 23)
G29 cost and value using G28: (2300, 23)


In [10]:
G29_alternative = G28.copy()

for u, v, data in G29_alternative.edges(data=True):
    if u == "Student 38 Grade 10"  and v == "MAT30-3":
        data['weight'] = 0
flow_dict28_alternative = nx.max_flow_min_cost(G29_alternative, "source", "sink")
section3_students29_alternative = set()
section5_student29_alternative = set()
# compare the flow values
for student, all_flow in flow_dict28_alternative.items():
    if student == "source" or student == "sink":
        continue
    for section, flow_value in all_flow.items():
        if section[:3] == "MAT":
            if flow_value > 0:
                if section[6] == "3":
                    section3_students29_alternative.add(student)
                    # print(student, section, flow_value)
                else:
                    section5_student29_alternative.add(student)
                # print(student, section, flow_value)

print(section3_students29_alternative)
print(section5_student29_alternative)

print("Ideally, this should be the same as G28")
print("difference with g29", section5_student29_alternative.difference(section5_student29))
print("differnce with g28", section5_student29_alternative.difference(section5_student28))

print("Cost of flow is the same")
print("G28 cost:", nx.cost_of_flow(G28, flow_dict28))
print("G29 cost:", nx.cost_of_flow(G29, flow_dict29))
print("G29 alternative cost:", nx.cost_of_flow(G29_alternative, flow_dict28_alternative))

print("Value of flow G28", value_of_flow(flow_dict28, G28))
print("Value of flow G29", value_of_flow(flow_dict29, G29))
print("Value of flow G29 alternative", value_of_flow(flow_dict28_alternative, G29_alternative))

{'Student 32 Grade 9', 'Student 45 Grade 9', 'Student 22 Grade 9', 'Student 50 Grade 9', 'Student 27 Grade 9', 'Student 19 Grade 9', 'Student 23 Grade 9', 'Student 33 Grade 9', 'Student 38 Grade 9', 'Student 26 Grade 9', 'Student 36 Grade 9', 'Student 43 Grade 9'}
{'Student 18 Grade 9', 'Student 34 Grade 9', 'Student 1 Grade 9', 'Student 31 Grade 9', 'Student 14 Grade 9', 'Student 12 Grade 9', 'Student 41 Grade 9', 'Student 17 Grade 9', 'Student 49 Grade 9', 'Student 24 Grade 9', 'Student 10 Grade 9'}
Ideally, this should be the same as G28
difference with g29 set()
differnce with g28 set()
Cost of flow is the same
G28 cost: 2300
G29 cost: 1090
G29 alternative cost: 2300
Value of flow G28 (2300, 23)
Value of flow G29 (1090, 23)
Value of flow G29 alternative (2300, 23)


In [11]:
section3_students29.difference(section3_students28)

set()

In [12]:
section3_students28.difference(section3_students29)

set()

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# create a pandas series of g28_edges

g28_edges_series = pd.Series(g28_edges)
g28_edges_series

g29_edges = G29.edges(data=True)
g29_edges_series = pd.Series(g29_edges)

# compare each entry in the two series
for i in range(len(g28_edges_series)):
    if g28_edges_series[i] != g29_edges_series[i]:
        print(f"Edge {i} in G28: {g28_edges_series[i]}")
        print(f"Edge {i} in G29: {g29_edges_series[i]}")
        print("Difference found!")
    else:
        print(f"Edge {i} in G28 and G29 are the same.")

df = pd.DataFrame(g28_edges_series, columns=["G28 Edges"])
df["G29 Edges"] = g29_edges_series

Edge 0 in G28 and G29 are the same.
Edge 1 in G28 and G29 are the same.
Edge 2 in G28 and G29 are the same.
Edge 3 in G28 and G29 are the same.
Edge 4 in G28 and G29 are the same.
Edge 5 in G28 and G29 are the same.
Edge 6 in G28 and G29 are the same.
Edge 7 in G28 and G29 are the same.
Edge 8 in G28 and G29 are the same.
Edge 9 in G28 and G29 are the same.
Edge 10 in G28 and G29 are the same.
Edge 11 in G28 and G29 are the same.
Edge 12 in G28 and G29 are the same.
Edge 13 in G28 and G29 are the same.
Edge 14 in G28 and G29 are the same.
Edge 15 in G28 and G29 are the same.
Edge 16 in G28 and G29 are the same.
Edge 17 in G28 and G29 are the same.
Edge 18 in G28 and G29 are the same.
Edge 19 in G28 and G29 are the same.
Edge 20 in G28 and G29 are the same.
Edge 21 in G28 and G29 are the same.
Edge 22 in G28 and G29 are the same.
Edge 23 in G28 and G29 are the same.
Edge 24 in G28 and G29 are the same.
Edge 25 in G28: ('Student 1 Grade 9', 'MAT10-5', {'weight': 100})
Edge 25 in G29: ('S

In [14]:
df

,G28 Edges,G29 Edges
0,"(source, Student 1 Grade 9, {'capacity': 1})","(source, Student 1 Grade 9, {'capacity': 1})"
1,"(source, Student 10 Grade 9, {'capacity': 1})","(source, Student 10 Grade 9, {'capacity': 1})"
2,"(source, Student 12 Grade 9, {'capacity': 1})","(source, Student 12 Grade 9, {'capacity': 1})"
3,"(source, Student 14 Grade 9, {'capacity': 1})","(source, Student 14 Grade 9, {'capacity': 1})"
4,"(source, Student 17 Grade 9, {'capacity': 1})","(source, Student 17 Grade 9, {'capacity': 1})"
5,"(source, Student 18 Grade 9, {'capacity': 1})","(source, Student 18 Grade 9, {'capacity': 1})"
6,"(source, Student 19 Grade 9, {'capacity': 1})","(source, Student 19 Grade 9, {'capacity': 1})"
7,"(source, Student 22 Grade 9, {'capacity': 1})","(source, Student 22 Grade 9, {'capacity': 1})"
8,"(source, Student 23 Grade 9, {'capacity': 1})","(source, Student 23 Grade 9, {'capacity': 1})"
9,"(source, Student 24 Grade 9, {'capacity': 1})","(source, Student 24 Grade 9, {'capacity': 1})"


In [16]:
print("G28", nx.cost_of_flow(G28, flow_dict28))
print("G29", nx.cost_of_flow(G29, flow_dict29))

print(nx.cost_of_flow(G28, flow_dict28_alternative))
print(nx.cost_of_flow(G28, flow_dict28))

G28 2300
G29 1090
2300
2300
